# Homework 4

**Bob Ding**

**zd26**

## Question 1

**Course mini-project thinking, proposal and development. Continue progressing on this. Email &/or talk to MW and TA Jiurui as convenient (some of you are already clearly moving along well here ...). In the near future, you will be required to submit and be assessed on a 2 page outline of your project.**

Done

## Question 2

**Review the DLM specification and forward filtering code for univariate DLMs in the course Matlab code using the Sales:Index time series example. There are a details of model and prior specification that we will move into next week, and it is critical that you are on top of the
basic model structuring and sequential analysis technicalities prior to these next steps. One key aspect of this is that model specification of evolution variance matrices Wt uses discount factors based on the component discount specification of P&W Section 4.3.6, p131, and as in course slides that we will review next week. Read and digest that material in P&W.**

**The code creates an example DLM for the Sales:Index analysis– a DLM with a trend component, a dynamic regression component, and a (Fourier) seasonal component. It then performs and summarizes forward filtering. Explore the model analysis as specified for the Sales:Index data as exemplified in class. Rerun the example and make sure you are really on top of the summaries of analysis produced. And, play with changes to the model specification and explore how summary “results” change. We will revisit this next week, and then in the next homework, with deeper investigation of issues of model choice and comparison.**

Done



## Question 3

**Derive/reproduce the Bayesian filtering theory that is at the core of the Kalman filtering (+variance learning) equations: P&W Section 4.3.1 and 4.3.2, and as reviewed in class and summarized in class slides. This an extension (to dynamic contexts, and with the state evolution prior to posterior update) of multivariate normal/inverse-gamma Bayesian theory; “simply”... but key and critical. Working through the details will help you fix ideas (as well as become comfortable with the notation). Working through some of the code will help too**

**Proof**



In [6]:
from IPython.display import Latex

In [7]:
%%latex
Suppose we have that

\begin{align*}
    y_{t} &= F_t^T\theta_{t-1} + \nu_t\\
    \theta_{t} &= G_t\theta_{t-1} + w_t\\\\
    \nu_t &\sim N(0, v_t)\\
    w_t &\sim N(0, W_t)\\ 
    \theta_t | \mathcal{D}_{t-1} &\sim N(a_t, R_t) = N(G_ta_{t-1}, G_tC_{t-1}G_t^T + W_t)\\
    y_t | \mathcal{D}_{t-1} &\sim N(f_t, q_t) = N(F_t^Ta_t, F_t^TR_tF_t + v_t)\\
    \theta_t | \mathcal{D}_{t} &\sim N(m_{t}, C_t)\\
\end{align*}

Our aim is to find out what is the value of $m_t, C_t$ and how does it correspond to  $m_{t-1}, C_{t_1}$

\begin{align*}
    \mathbb{P}(\theta_t | \mathcal{D}_t) &= \frac{\mathbb{P}(y_t | \theta_t, \mathcal{D}_{t-1})\mathbb{P}(\theta_t | \mathcal{D}_{t-1})}{\mathbb{P}(y_t | \mathcal{D}_{t-1})}\\
    &\propto \mathbb{P}(y_t | \theta_t, \mathcal{D}_{t-1})\mathbb{P}(\theta_t | \mathcal{D}_{t-1})\\
    &= N(F_t^T \theta_t, v_t)N(a_t, R_t)\\
    &\propto \exp\left\{-\frac{1}{2v_t}(y_t - F_t^T\theta_t)^2  \right\}\exp\left\{-\frac{1}{2}(\theta_t - a_t)^TR_t^{-1}(\theta_t - a_t)  \right\}\\
    &\propto \exp\left\{-\frac{1}{2} \left[\frac{1}{v_t} (y_t - F_t^T\theta_t)^2  +(\theta_t - a_t)^TR_t^{-1}(\theta_t - a_t) \right] \right\}\\
    &\propto \exp\left\{-\frac{1}{2} \left[\frac{1}{v_t}\theta_t^TF_t F_t^T\theta_t - 2\theta_t^T F_t y_t(v_t)^{-1} + \theta_t^TR_t^{-1}\theta_t -2\theta_t^T R_t^{-1}a_t \right] \right\}\\
    &\propto \exp\left\{-\frac{1}{2} \left[ \theta_t^T(\frac{1}{v_t}F_tF_t^T + R_{t}^{-1}) \theta_t-2\theta_t^T(F_ty_t(v_t)^{-1} + R_t^{-1}a_t) \right] \right\}\\
    &\sim N\left( \left( \frac{1}{v_t}F_tF_t^T + R_{t}^{-1} \right)^{-1}\left( F_ty_t(v_t)^{-1} + R_t^{-1}a_t  \right) ,  \left( \frac{1}{v_t}F_tF_t^T + R_{t}^{-1} \right)^{-1} \right)\\
    &= N\left( \left(  R_t - \frac{R_tF_tF_t^TR_t^T }{F_t^TR_tF_t + v_t}  \right)\left( F_ty_t(v_t)^{-1} + R_t^{-1}a_t  \right) ,  R_t - \frac{R_tF_tF_t^TR_t^T }{F_t^TR_tF_t + v_t} \right)\\
    &= N\left( a_t + \frac{R_tF_t}{F_t^TR_tF_t + v_t}(y_t - F_t^Ta_t) ,  R_t - \frac{R_tF_tF_t^TR_t^T }{F_t^TR_tF_t + v_t} \right)\\
    &= N\left( a_t + A_te_t ,  R_t - A_tA_t^Tq_t \right)\\
    &= N\left( m_t, C_t \right)\\
\end{align*}
    
Prior for time t:
The conditional prior is calculated below.
\begin{align*}
\begin{aligned}
E\left[\theta_{t} | v, \mathcal{D}_{t-1}\right] &=E\left[\mathbf{G}_{t} \theta_{t-1}+\omega_{t} | v, \mathcal{D}_{t-1}\right] \\
&=\mathbf{G}_{t} E\left[\theta_{t-1} | v, \mathcal{D}_{t-1}\right]+E\left[\omega_{t} | v, \mathcal{D}_{t-1}\right] \\
&=\mathbf{G}_{t} \mathbf{m}_{t-1}+0 \\
&=\mathbf{G}_{t} \mathbf{m}_{t-1}+0 \\
&=V\left[\mathbf{G}_{t} \theta_{t-1}+\omega_{t} | v, \mathcal{D}_{t-1}\right] \\
&=\frac{v}{s_{t} V\left[\theta_{t-1}\left|v, \mathcal{D}_{t-1}\right| \mathbf{G}_{t}^{T}+V\left[\omega_{t} | \mathcal{D}_{t-1}\right]\right.} \\
&=\frac{v}{s_{t-1}}\left(\mathbf{G}_{t} \mathbf{C}_{t-1} \mathbf{G}_{t}^{T}+\mathbf{W}_{t}\right) \\
\Rightarrow \theta_{t} | v, \mathcal{D}_{t-1} & \sim N\left(\mathbf{G}_{t} \mathbf{m}_{t-1}, \frac{v}{s_{t-1}}\left(\mathbf{G}_{t} \mathbf{C}_{t-1} \mathbf{G}_{t}^{T}+\mathbf{W}_{t}\right)\right)
\end{aligned}
\end{align*}
This conditional prior implies, with $\left(v | \mathcal{D}_{t-1}\right),$ the following marginal prior for $\theta_{t}$
\begin{align*}
\theta_{t} | v, \mathcal{D}_{t-1} \sim N\left(\mathbf{G}_{t} \mathbf{m}_{t-1}, \frac{v}{s_{t-1}}\left(\mathbf{G}_{t} \mathbf{C}_{t-1} \mathbf{G}_{t}^{T}+\mathbf{W}_{t}\right)\right)
\end{align*}
\begin{align*}
\begin{aligned}
& v | \mathcal{D}_{t-1} \sim n_{t-1} s_{t-1} / \chi_{n_{t-1}}^{2} \\
\Rightarrow & \theta_{t} | \mathcal{D}_{t-1} \sim T_{n_{t}-1}\left(\mathbf{G}_{t} \mathbf{m}_{t-1},\left(\mathbf{G}_{t} \mathbf{C}_{t-1} \mathbf{G}_{t}^{T}+\mathbf{W}_{t}\right)\right)
\end{aligned}
\end{align*}
                        
The conditional predictive distribution is calculated below.

\begin{align*}
\begin{aligned}
E\left[y_{t} | v, \mathcal{D}_{t-1}\right] &=E\left[\mathbf{F}_{t}^{T} \theta_{t}+\nu_{t} | v, \mathcal{D}_{t-1}\right] \\
&=\mathbf{F}_{t}^{T} \mathbf{a}_{t} \\
V\left[y_{t} | v, \mathcal{D}_{t-1}\right] &=V\left[\mathbf{F}_{t}^{T} \theta_{t}+\nu_{t} | v, \mathcal{D}_{t-1}\right] \\
&=\mathbf{F}_{t}^{T} \frac{v}{s_{t-1}} \mathbf{R}_{t} \mathbf{F}_{t}+v \\
&=\frac{v}{s_{t-1}} q_{t} \\
&=\frac{v}{s_{t-1}} q_{t} \\
\Rightarrow y_{t} | v, \mathcal{D}_{t-1} & \sim N\left(\mathbf{F}_{t}^{T} \mathbf{a}_{t}, \frac{v}{s_{t-1}} q_{t}\right)
\end{aligned}
\end{align*}
                        
This conditional predictive distribution, with $v | \mathcal{D}_{t-1},$ implies the following marginal predictive distribution.
\begin{align*}
\begin{aligned}
y_{t} | v, \mathcal{D}_{t-1} & \sim N\left(\mathbf{F}_{t}^{T} \mathbf{a}_{t}, \frac{v}{s_{t-1}} q_{t}\right) \\
v | \mathcal{D}_{t-1} & \sim n_{t-1} s_{t-1} / \chi_{n_{t-1}}^{2} \\
\Rightarrow y_{t} | \mathcal{D}_{t-1} & \sim T_{n_{t-1}}\left(\mathbf{F}_{t}^{T} \mathbf{a}_{t}, q_{t}\right) \\
=& T_{n_{t-1}}\left(f_{t}, q_{t}\right)
\end{aligned}
\end{align*}
Posterior at time t:
The conditional posterior will be calculated first. Then, the marginal posterior will be calculated.
$p\left(\theta_{t} | v, \mathcal{D}_{t}\right) \propto p\left(y_{t} | \theta_{t}\right) p\left(\theta_{t} | v, \mathcal{D}_{t-1}\right)$
Similar to the ordinary Kalman filter, the posterior mean and variance above will be shown to be equivalent to those in the class slides.
First, the variance will be shown equivalent. The binomial inverse theorem is re-used here.
\begin{align*}
\begin{aligned}
\left(\left[\frac{v \mathbf{R}_{t}}{s_{t-1}}\right]^{-1}+\mathbf{F}_{t} v^{-1} \mathbf{F}_{t}^{T}\right)^{-1} &=\frac{v \mathbf{R}_{t}}{s_{t-1}}-\frac{v \mathbf{R}_{t}}{s_{t-1}} \mathbf{F}_{t}\left(v+\mathbf{F}_{t}^{T}\left[\frac{v \mathbf{R}_{t}}{s_{t-1}}\right] \mathbf{F}_{t}\right)^{-1} \mathbf{F}_{t}\left(\frac{v \mathbf{R}_{t}}{s_{t-1}}\right) \\
&=\frac{v \mathbf{R}_{t}}{s_{t-1}}-\frac{v \mathbf{R}_{t}}{s_{t-1}} \mathbf{F}_{t}\left[\frac{v}{s_{t-1}}\left(s_{t-1}-\mathbf{F}_{t}^{T} \mathbf{R}_{t} \mathbf{F}_{t}\right)\right]^{-1} \mathbf{F}_{t}^{T}\left(\frac{v \mathbf{R}_{t}}{s_{t-1}}\right) \\
&=\frac{v}{s_{t-1}}\left[\mathbf{R}_{t}-\mathbf{R}_{t} \mathbf{F}_{t}\left(s_{t-1}-\mathbf{F}_{t}^{T} \mathbf{R}_{t} \mathbf{F}_{t}\right)^{-1} \mathbf{F}_{t}^{T} \mathbf{R}_{t}\right] \\
&=\frac{v}{s_{t-1}}\left[\mathbf{R}_{t}-q_{t} \mathbf{A}_{t} \mathbf{A}_{t}^{T}\right]
\end{aligned}
\end{align*}
Now, the mean will be shown to be equivalent. Note that $\left(\mathbf{R}_{t}^{-1}+\mathbf{F}_{t} s_{t-1}^{-1} \mathbf{F}_{t}^{T}\right) \mathbf{F}_{t} s_{t-1}^{-1}=\mathbf{R}_{t} \mathbf{F}_{t} q_{t}^{-1}$ with the
binomial inverse theorem and simplification.

                        $=\mathbf{m}_{\mathrm{t}}$
Thus, the conditional posterior is $\theta_{t} | v, \mathcal{D}_{t} \sim N\left(\mathbf{m}_{t}, \frac{v}{s_{t-1}}\left[\mathbf{R}_{t}-q_{t} \mathbf{A}_{t} \mathbf{A}_{t}^{T}\right]\right),$ which implies a marginal posterior
of $\theta_{t} | \mathcal{D}_{t} \sim T_{n_{t}}\left(\mathbf{m}_{\mathbf{t}}, \frac{s_{t}}{s_{t-1}}\left[\mathbf{R}_{t}-q_{t} \mathbf{A}_{t} \mathbf{A}_{t}^{T}\right]\right)$ with $v | \mathcal{D}_{t} \sim n_{t} s_{t} / \chi_{n_{t}}^{2}$

<IPython.core.display.Latex object>

## Question 4

**Consider a DLM with time $t$ observation variance $v_{t}$ known. The evolution model setup is summarised by the time $t-1$ posterior  \left({\theta}_{t-1} | {D}_{t-1}\right) \sim N\left({m}_{t-1}, {C}_{t-1}\right)$ that evolves through the state equation ${\theta}_{t}={G}_{t} {\theta}_{t-1}+{\omega}_{t}$ where ${\omega}_{t} \sim N\left({0},{W}_{t}\right)$ and ${\theta}_{t-1} \perp {\omega}_{t}$ Suppose now the special case in which:**

**$\cdot$ the model has a random walk state evolution, i.e., $\mathbf{G}_{t}=\mathbf{I}$ for all $t,$ and**
**$\cdot$ the evolution variance matrix at time $t$ is defined by the specific formula $\mathbf{W}_{t}=\epsilon \mathbf{C}_{t-1}$ where $\epsilon=(1-\delta) / \delta$ for some discount factor $\delta \in(0,1)$**

* (a) **Show how the "Kalman filter" update equations for prior:posterior analysis at time $t$ simplify in this special case.**

**Proof**

\begin{aligned}
        \theta_t | \mathcal{D}_{t-1} &\sim N(Gm_{t-1}, GC_{t-1}G^T + \epsilon C_{t-1})\\
        \theta_t | \mathcal{D}_t &\sim N(Gm_{t-1} + \frac{R_tF_t}{q_t}(y_t - F_t^TG_tm_{t-1}), R_t - \frac{R_tF_tF_t^TR_t^T}{q_t})\\
        &\sim N\left(m_{t-1}+ \frac{(1+\epsilon) C_{t-1}F_t}{(1+\epsilon)F_t^TC_{t-1}F_t + v_t}(y_t - F_t^Tm_{t-1}) , (1+\epsilon)C_{t-1} -(1+\epsilon)^2\frac{C_{t-1}F_tF_t^TC_{t-1}^T}{(1+\epsilon)F_t^TC_{t-1}F_t + v_t}  \right)\\
        &\sim N\left(m_{t-1}+ \frac{\frac{1}{\delta} C_{t-1}F_t}{\frac{1}{\delta}F_t^TC_{t-1}F_t + v_t}(y_t - F_t^Tm_{t-1}) , \frac{1}{\delta}C_{t-1} -\frac{1}{\delta^2}\frac{C_{t-1}F_tF_t^TC_{t-1}^T}{\frac{1}{\delta}F_t^TC_{t-1}F_t + v_t}  \right)\\
\end{aligned}


* (b) **Comment on the simplified structure and how it depends on the chosen/specified discount factor $\delta$**

**Proof**

Smaller the discount factor $\delta$ will leed to a bigger value of variance. Besides, the variance might even diverge as it increases constantly.

* (c) **Comment on the computational implications of this simplified structure.**

This is easier to compute as we don't need as many matrix multiplication

## Question 5

* (a)


\begin{aligned}
    C(\theta_t, \theta_{t-1} | D_{t-1}) &= E(\theta_{t-1}^TG_{t}\theta_{t-1}) - m_{t-1}^TG_tm_{t-1}\\
    &= (m_{t-1}^TG_{t}m_{t-1} + G_tC_{t-1}) - m_{t-1}^TG_tm_{t-1}\\
    &= G_tC_{t-1}\\
\end{aligned}

* (b)

From all the work we've done in p4, we know that the joint is jointly normal. Therefore, the marginal, by definition, is normal.

The parameters for the conditional distribution are calculated below using the conditional expectation and conditional variance formulas.
\begin{aligned}
\begin{array}{l}
{E\left[\theta_{t-1} | \theta_{t}, \mathcal{D}_{t-1}\right]=\mathbf{m}_{t-1}+\mathbf{C}_{t-1} \mathbf{G}_{t}^{T} \mathbf{R}_{t}^{-1}\left(\theta_{t}-\mathbf{a}_{t}\right)} \\
{V\left[\theta_{t-1} | \theta_{t}, \mathcal{D}_{t-1}\right]=\mathbf{C}_{t-1}-\mathbf{C}_{t-1} \mathbf{G}_{t}^{T} \mathbf{R}_{t}^{-1} \mathbf{G}_{t} \mathbf{C}_{t-1}}
\end{array}
\end{aligned}
Suppose $\mathbf{B}_{t-1}=\mathbf{C}_{t} \mathbf{G}_{t}^{T} \mathbf{R}_{t}^{-1} .$ Then

\begin{aligned}
E\left[\theta_{t-1} | \theta_{t}, \mathcal{D}_{t-1}\right] &=\mathbf{m}_{t-1}+\mathbf{B}_{t-1}\left(\theta_{t}-\mathbf{a}_{t}\right) \\
V\left[\theta_{t-1} | \theta_{t}, \mathcal{D}_{t-1}\right] &=\mathbf{C}_{t-1}-\mathbf{B}_{t-1} \mathbf{G}_{t} \mathbf{C}_{t-1} \\
&=\mathbf{C}_{t-1}-\mathbf{B}_{t-1} \mathbf{R}_{t} \mathbf{R}_{t}^{-1} \mathbf{G}_{t} \mathbf{C}_{t-1} \\
&=\mathbf{C}_{t-1}-\mathbf{B}_{t-1} \mathbf{R}_{t} \mathbf{B}_{t-1}^{T}
\end{aligned}

Thus, $p\left(\theta_{t-1} | \theta_{t}, \mathcal{D}_{t-1}\right) \sim N\left(\mathbf{m}_{t-1}+\mathbf{B}_{t-1}\left(\theta_{t}-\mathbf{a}_{t}\right), \mathbf{C}_{t-1}-\mathbf{B}_{t-1} \mathbf{R}_{t} \mathbf{B}_{t-1}^{T}\right)$

* (c)

$\begin{aligned} p\left(\theta_{t-1} | \theta_{t}, \mathcal{D}_{n}\right) &=p\left(\theta_{t-1} | \theta_{t}, \mathcal{D}_{t-1}\right) \\ &=N\left(\mathbf{m}_{t-1}+\mathbf{B}_{t-1}\left(\theta_{t}-\mathbf{a}_{t}\right), \mathbf{C}_{t-1}-\mathbf{B}_{t-1} \mathbf{R}_{t} \mathbf{B}_{t-1}^{T}\right) \end{aligned}$


* (d)

\begin{aligned}
p\left(\theta_{1: n} | \mathcal{D}_{n}\right) &=p\left(\theta_{n} | \mathcal{D}_{n}\right) p\left(\theta_{n-1} | \theta_{n}, \mathcal{D}_{n}\right) \ldots p\left(\theta_{2} | \theta_{3}, \mathcal{D}_{n}\right) p\left(\theta_{1} | \theta_{2}, \mathcal{D}_{n}\right) \\
&=p\left(\theta_{n} | \mathcal{D}_{n}\right)\left[\Pi_{i=1}^{n-1} p\left(\theta_{i} | \theta_{i+1}, \mathcal{D}_{n}\right)\right]
\end{aligned}

The full trajectory of states $p\left(\theta_{1: n} | \mathcal{D}_{n}\right)$ at any chosen time point $n$ is the product of $n-1$ previous retrospective distributions and the marginal $\left(\theta_{n} | \mathcal{D}_{n}\right) .$ This result allows the joint distribution to be derived from retrospective distributions, if the marginal $\left(\theta_{n} | \mathcal{D}_{n}\right)$ is known.

* (e)

\begin{aligned} 
C\left[\theta_{t}, \theta_{t-1} | \mathcal{D}_{t-1}\right] 
&=\mathbf{G}_{t} \mathbf{C}_{t-1} \\ 
&=\mathbf{C}_{t-1} 
\end{aligned}



## Question 6

Done